In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (5,386 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 122349 files and di

### 필수과제 2
- 고객데이터를 올릴 것
- 해당 고객데이터를 기반으로 여러분들이 가설을 세워서 어떤 컬럼을 사용할 것인지 정리하고
- 해당 컬럼을 기반으로 kmeans를 진행하여서 분석에 대한 인사이트를 정리해 주세요!
- 군집에 대해서 몇 개를 선정하는지에 대한 질문의 답변은 위에 적어놨으니
- 가능하신 분은 구글링을 통해서 해당 지표도 만들어 보시면 좋을 것 같습니다.

- 인사이트입니다.
- 기존에 데이터가 어떤 패턴을 보였고
- 우리가 선정한 컬럼을 통해서 만들어진 군집 ( 고객군 ) 어떤 패턴을 보이는지

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [71]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale,MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [54]:
customer=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/NHcustomer.csv')

In [55]:
ct=customer[['성별','연령구간','투자성향','고객자산구간','총투자기간','주거래상품군','LIFESTAGE','고객등급','주거래업종구분']]

In [56]:
ct

,성별,연령구간,투자성향,고객자산구간,총투자기간,주거래상품군,LIFESTAGE,고객등급,주거래업종구분
0,1,4,99,1,6,2,3,3,16
1,1,6,4,4,6,2,5,2,1
2,2,7,4,3,6,2,5,5,9
3,2,6,4,4,6,8,5,3,16
4,1,5,2,2,5,2,5,5,16
...,...,...,...,...,...,...,...,...,...
9995,1,9,4,2,2,2,9,5,16
9996,2,2,3,1,1,2,2,9,16
9997,2,7,4,2,2,2,5,5,16
9998,1,5,99,5,2,2,5,3,5


In [78]:
print(ct['고객등급'].unique())
ct['고객등급'].value_counts()

[ 3  2  5  4  9  1 99]


9     3366
5     2687
4     2298
3     1184
2      345
1       83
99      37
Name: 고객등급, dtype: int64

In [77]:
print(ct['총투자기간'].unique())    # 투자기간을 기준으로 분류하자
ct['총투자기간'].value_counts()

[6 5 4 3 2 1]


3    4271
2    3315
4     992
1     962
5     312
6     148
Name: 총투자기간, dtype: int64

In [76]:
print(ct['투자성향'].unique())
ct['투자성향'].value_counts()

[99  4  2  3  5  1  9  0]


99    3687
4     1858
3     1669
2     1467
5      935
1      210
0      171
9        3
Name: 투자성향, dtype: int64

In [74]:
ct['주거래상품군'].unique()
ct['주거래상품군'].value_counts()   # 주거래상품군 컬럼은 2에 몰려있다.

2     9659
3      261
1       34
15      10
10       9
8        8
5        7
9        5
11       3
6        3
12       1
Name: 주거래상품군, dtype: int64

### 컬럼 선택

In [57]:
for i in list(ct.columns):
  print('-----------------',i ,'mean 비교')
  print(ct.groupby(['총투자기간'])[i].mean())

----------------- 성별 mean 비교
총투자기간
1    1.457380
2    1.433786
3    1.384922
4    1.337702
5    1.339744
6    1.351351
Name: 성별, dtype: float64
----------------- 연령구간 mean 비교
총투자기간
1    3.644491
2    4.444947
3    4.790213
4    5.318548
5    6.032051
6    6.520270
Name: 연령구간, dtype: float64
----------------- 투자성향 mean 비교
총투자기간
1    50.565489
2    42.289593
3    34.072583
4    38.217742
5    28.394231
6    28.168919
Name: 투자성향, dtype: float64
----------------- 고객자산구간 mean 비교
총투자기간
1    1.422037
2    2.234992
3    2.541793
4    3.050403
5    3.506410
6    3.770270
Name: 고객자산구간, dtype: float64
----------------- 총투자기간 mean 비교
총투자기간
1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
Name: 총투자기간, dtype: float64
----------------- 주거래상품군 mean 비교
총투자기간
1    2.028067
2    2.047360
3    2.059471
4    2.029234
5    2.246795
6    2.256757
Name: 주거래상품군, dtype: float64
----------------- LIFESTAGE mean 비교
총투자기간
1    3.675676
2    4.275415
3    4.571295
4    4.938508
5    5.326923
6    5.614865
Name

### 전체 컬럼으로 kmeans

In [79]:
# 스케일링 x
ct_x=ct[['성별','연령구간','투자성향','고객자산구간','주거래상품군','LIFESTAGE','고객등급','주거래업종구분']]
kmeans=KMeans(n_clusters=6, init='k-means++', max_iter=300, random_state=111)
kmeans.fit(ct_x)
ct['cluster'] = kmeans.labels_
ct.groupby(['cluster'])['성별'].count()

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-79-240527827dca>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct['cluster'] = kmeans.labels_


cluster
0    2285
1    2161
2      23
3    4014
4      14
5    1503
Name: 성별, dtype: int64

### 평균 차이가 크지 않은 컬럼 선택
- 성별
- 주거래상품군
- 주거래업종구분

In [69]:
# 스케일링 x
ct_x=ct[['성별','주거래상품군','주거래업종구분']]
kmeans=KMeans(n_clusters=6, init='k-means++', max_iter=300, random_state=111)
kmeans.fit(ct_x)
ct['cluster'] = kmeans.labels_
ct.groupby(['cluster'])['성별'].count()

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-69-8532adab8bc9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct['cluster'] = kmeans.labels_


cluster
0    1688
1     874
2    3510
3    2087
4     476
5    1365
Name: 성별, dtype: int64

In [72]:
ct_x=ct[['성별','주거래상품군','주거래업종구분']]
kmeans=KMeans(n_clusters=6, init='k-means++', max_iter=300, random_state=111)

# 스케일링
mm_scaler=MinMaxScaler()
mm_scaler.fit(ct_x)
ct_xx=mm_scaler.transform(ct_x)
df_ct_xx= pd.DataFrame(ct_xx, columns = ct_x.columns)

kmeans.fit(df_ct_xx)
ct['cluster'] = kmeans.labels_
ct.groupby(['cluster'])['성별'].count()

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-72-abef102b73f1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct['cluster'] = kmeans.labels_


cluster
0    1326
1     839
2    2563
3    1354
4    1822
5    2096
Name: 성별, dtype: int64

- 전체 컬럼을 사용했을 때보다는 클러스터링이 잘 됐다고 볼 수 있다.
- MinMax스케일링 후의 분류가 더 안 된 것 같다.
